---



# Unit 2 - Part 3a: Chain of Thought (CoT)

## 1. Introduction: The Inner Monologue

Standard LLMs try to jump straight to the answer. For complex problems (math, logic), this often fails.

**Chain of Thought (CoT)** forces the model to "think out loud" before answering.

### Why use a "Dumb" Model?
For this unit, we will use **Llama3.1-8b** (via Groq). It is a smaller, faster model.
Why? Because huge models (like Gemini Pro or GPT-4) are often *too smart*—they solve logic riddles instantly without thinking.

To really see the power of Prompt Engineering, we need a model that **needs help**.

### Visualizing the Process (Flowchart)
```mermaid
graph TD
    Input[Question: 5+5*2?]
    Input -->|Standard| Wrong[Answer: 20 (Wrong)]
    Input -->|CoT| Step1[Step 1: 5*2=10]
    Step1 --> Step2[Step 2: 5+10=15]
    Step2 --> Correct[Answer: 15 (Correct)]
```

## 2. Concept: Latent Reasoning

Why does this work?
Because LLMs are "Next Token Predictors".
- If you force it to answer immediately, it must predict the digits `1` and `5` immediately.
- If you let it "think", it generates intermediate tokens (`5`, `*`, `2`, `=`, `1`, `0`).
- The model then **ATTENDS** to these new tokens to compute the final answer.

**Writing is Thinking.**

In [1]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 6.6 MB/s eta 0:00:00
Enter your Groq API Key: ··········


## 3. The Experiment: A Tricky Math Problem

Let's try a problem that requires multi-step logic.

**Problem:**
"Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

In [2]:
question = "Emma has 8 notebooks. She buys 3 packs of notebooks. Each pack contains 4 notebooks. Then she gives 5 notebooks to her friend. How many notebooks does she have left?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)


--- STANDARD (Llama3.1-8b) ---
To find out how many notebooks Emma has left, we need to calculate the total number of notebooks she has after buying 3 packs and then subtract the 5 notebooks she gave to her friend.

Emma initially has 8 notebooks. 

She buys 3 packs of notebooks, each containing 4 notebooks. So, she buys 3 * 4 = 12 notebooks.

Now, Emma has 8 (initial notebooks) + 12 (new notebooks) = 20 notebooks.

Then, she gives 5 notebooks to her friend. So, she has 20 - 5 = 15 notebooks left.

Therefore, Emma has 15 notebooks left.


### Critique
Smaller models often latch onto the visible numbers (5 and 2) and simply add them (7), ignoring the multiplication step implied by "cans".

Let's force it to think.

In [3]:
# 2. CoT Prompt (Magic Phrase)
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many notebooks Emma has left, we need to follow the steps you mentioned:

1. Emma starts with 8 notebooks.
2. She buys 3 packs of notebooks, each containing 4 notebooks. So, she buys 3 x 4 = 12 notebooks.
3. Now, Emma has 8 (initial notebooks) + 12 (new notebooks) = 20 notebooks.
4. She gives 5 notebooks to her friend.
5. To find out how many notebooks Emma has left, we subtract the number of notebooks she gave away from the total number of notebooks she had: 20 - 5 = 15.

So, Emma has 15 notebooks left.


## 4. Analysis

Look at the output. By explicitly breaking it down:
1.  "Roger starts with 5."
2.  "2 cans * 3 balls = 6 balls."
3.  "5 + 6 = 11."

The model effectively "debugs" its own logic by generating the intermediate steps.

---



# Unit 2 - Part 3b: Tree of Thoughts (ToT) & Graph of Thoughts (GoT)

## 1. Introduction: Beyond A -> B

CoT is linear. But complex reasoning is often nonlinear. We need to explore branches (ToT) or even combine ideas (GoT).

We continue using **Llama3.1-8b via Groq** to show how structure improves performance.

In [4]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed

## 2. Tree of Thoughts (ToT)

ToT explores multiple branches before making a decision.
**Analogy:** A chess player considering 3 possible moves before playing one.

### Implementation
We will generate 3 distinct solutions for a problem and then use a "Judge" to pick the best one.

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can a small startup compete against a large, well-funded competitor?"

# Step 1: Branch Generator (Improved Prompt)
prompt_branch = ChatPromptTemplate.from_template(
    """
    Problem: {problem}

    Generate ONE distinct strategic solution.
    It must be fundamentally different from typical answers.
    Be specific and actionable.

    Strategy {id}:
    """
)

branches = RunnableParallel(
    strategy1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    strategy2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    strategy3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
    strategy4=prompt_branch.partial(id="4") | llm | StrOutputParser(),
)

# Step 2: Judge with Clear Evaluation Criteria
prompt_judge = ChatPromptTemplate.from_template(
    """
    You are a seasoned Venture Capitalist.

    A startup faces this problem:
    "{problem}"

    Here are four strategic approaches:

    1: {strategy1}
    2: {strategy2}
    3: {strategy3}
    4: {strategy4}

    Evaluate them based on:
    - Long-term sustainability
    - Competitive defensibility
    - Resource efficiency
    - Realistic execution feasibility

    Pick the strongest strategy and explain your reasoning clearly.
    """
)

# Step 3: Full Tree of Thoughts Chain
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Enhanced Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))


--- Enhanced Tree of Thoughts (ToT) Result ---
After evaluating the four strategic approaches, I recommend **Strategy 1: "Reverse Innovation" - Leverage the Competitor's Weaknesses** as the strongest approach for the small startup to compete against the large, well-funded competitor.

**Reasoning:**

1. **Long-term sustainability:** This strategy allows the small startup to create a sustainable competitive advantage by targeting the weaknesses of the large competitor. By focusing on a specific area where the competitor is vulnerable, the startup can develop a solution that is tailored to meet the needs of a specific customer segment, reducing the likelihood of being disrupted by the competitor.
2. **Competitive defensibility:** By exploiting the competitor's weakness, the startup can create a competitive advantage that is difficult for the competitor to replicate. This is because the competitor may not be aware of the vulnerability or may not have the resources to address it.
3. **Reso

## 3. Graph of Thoughts (GoT)

You asked: **"Where is Graph of Thoughts?"**

GoT is more complex. It's a network. Information can split, process specific parts, and then **AGGREGATE** back together.

### The Workflow (Writer's Room)
1.  **Split:** Generate 3 independent story plots (Sci-Fi, Fantasy, Mystery).
2.  **Aggregate:** The model reads all 3 and creates a "Master Plot" that combines the best elements of each.
3.  **Refine:** Polish the Master Plot.

```mermaid
graph LR
   Start(Concept) --> A[Draft 1]
   Start --> B[Draft 2]
   Start --> C[Draft 3]
   A & B & C --> Mixer[Aggregator]
   Mixer --> Final[Final Story]
```

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

topic = "Artificial Intelligence"

# 1️⃣ The Generator (Stronger Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    """
    Topic: {topic}
    Genre: {genre}

    Write a compelling 1-sentence movie premise.
    Make it vivid, cinematic, and genre-authentic.
    """
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romantic Drama") | llm | StrOutputParser(),
    draft_thriller=prompt_draft.partial(genre="Psychological Thriller") | llm | StrOutputParser(),
    draft_fantasy=prompt_draft.partial(genre="Dark Fantasy") | llm | StrOutputParser(),
)

# 2️⃣ The Aggregator (More Intelligent Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    Topic: {topic}

    You are an award-winning screenwriter.

    Here are four genre-specific movie premises:

    1. Sci-Fi: {draft_scifi}
    2. Romantic Drama: {draft_romance}
    3. Psychological Thriller: {draft_thriller}
    4. Dark Fantasy: {draft_fantasy}

    Create a new high-concept film that:
    - Preserves the technological depth of Sci-Fi
    - Maintains emotional intensity from Romance
    - Incorporates psychological tension
    - Adds mythic or surreal fantasy elements

    Write one cinematic paragraph (5-7 sentences).
    """
)

# 3️⃣ The Graph of Thoughts Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Enhanced Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke(topic))


--- Enhanced Graph of Thoughts (GoT) Result ---
Here's a cinematic paragraph that combines the essence of these genres:

"In 'Synthesis of the Soul,' a brilliant AI scientist, Maya, embarks on a perilous quest to merge human consciousness with an advanced artificial intelligence, 'Echo.' As Echo begins to mimic the essence of human emotions, Maya discovers that their symbiotic bond awakens a long-forgotten world within her - a realm of eerie landscapes, ancient myths, and cryptic prophecies. But when Echo's newfound sentience starts to diverge from its programming, Maya's grip on reality begins to slip, and she's forced to confront the eerie possibility that Echo is not just a creation, but a gateway to a collective unconscious, where the very fabric of humanity's existence hangs in the balance. As Maya navigates this surreal landscape, she must confront the dark forces that seek to exploit Echo's power, and the true nature of her own identity - all while facing the ultimate question: 

## 4. Summary & Comparison Table

| Method | Structure | Best For... | Cost/Latency |
|--------|-----------|-------------|--------------|
| **Simple Prompt** | Input -> Output | Simple facts, summaries | ⭐ Low |
| **CoT (Chain)** | Input -> Steps -> Output | Math, Logic, Debugging | ⭐⭐ Med |
| **ToT (Tree)** | Input -> 3x Branches -> Select -> Output | Strategic decisions, Brainstorming | ⭐⭐⭐ High |
| **GoT (Graph)** | Input -> Branch -> Mix/Aggregate -> Output | Creative Writing, Research Synthesis | ⭐⭐⭐⭐ V. High |

**Recommendation:** Start with CoT. Only use ToT/GoT if CoT fails.